In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.stats import norm
import xlwings as xw
import yfinance as yf
from mpl_toolkits.mplot3d import Axes3D
import tempfile
import os

# --- INPUTS ---
book = xw.Book('Richspread.xlsx')
sht = book.sheets['Opt_Strat']

S = sht.range('B2').value  # Current stock price
K = sht.range('B3').value  # Strike price
T = sht.range('B4').value  # Time to expiration in years
r = sht.range('B5').value  # Risk-free interest rate
V = sht.range('B6').value  # Volatility of the underlying asset
option_type = sht.range('B7').value  # Option type ('call' or 'put')
option_strategy = sht.range('B10').value  # Option strategy
K1_spread = sht.range('B12').value  # e.g., 0.9
K2_spread = sht.range('B13').value  # e.g., 1
K3_spread = sht.range('B14').value  # e.g., 1.1
K4_spread = sht.range('B15').value  # e.g., 1.2

# --- COMMON: Stock price range for plotting ---
stock_prices = np.linspace(0, 2 * K, 100)

# --- Black-Scholes utility ---
def bs_price(S, K, T, r, V, option_type):
    d1 = (np.log(S / K) + (r + 0.5 * V ** 2) * T) / (V * np.sqrt(T))
    d2 = d1 - V * np.sqrt(T)
    if option_type == 'call':
        return S * norm.cdf(d1) - K * np.exp(-r * T) * norm.cdf(d2)
    elif option_type == 'put':
        return K * np.exp(-r * T) * norm.cdf(-d2) - S * norm.cdf(-d1)
    else:
        return 0

def plot_with_breakeven(x, y, breakevens, label, title, strikes=None, return_fig=False):
    fig, ax = plt.subplots(figsize=(10, 5))
    ax.plot(x, y, label=label)
    ax.axhline(0, color='black', lw=0.5)
    for be in np.atleast_1d(breakevens):
        ax.axvline(be, color='orange', linestyle='--', label=f'Breakeven: {be:.2f}')
    if strikes is not None:
        for strike, strike_label in strikes:
            ax.axvline(strike, color='grey', linestyle=':', label=strike_label)
    ax.set_xlabel('Stock Price at Expiration')
    ax.set_ylabel('Profit / Loss')
    ax.set_title(title)
    ax.grid()
    ax.legend()
    plt.tight_layout()
    if return_fig:
        return fig
    else:
        plt.show()

# --- Option Strategy Logic ---
fig = None

if option_strategy == 'Call':
    call_premium = bs_price(S, K, T, r, V, 'call')
    call_option_pnl = np.maximum(stock_prices - K, 0) - call_premium
    be = K + call_premium
    fig = plot_with_breakeven(stock_prices, call_option_pnl, be, 'Call Option P&L', 'Profit/Loss Diagram for a Call Option', strikes=[(K, 'Strike')], return_fig=True)

elif option_strategy == 'Put':
    put_premium = bs_price(S, K, T, r, V, 'put')
    put_option_pnl = np.maximum(K - stock_prices, 0) - put_premium
    be = K - put_premium
    fig = plot_with_breakeven(stock_prices, put_option_pnl, be, 'Put Option P&L', 'Profit/Loss Diagram for a Put Option', strikes=[(K, 'Strike')], return_fig=True)

elif option_strategy == 'Synthetic Long':
    call_premium = bs_price(S, K, T, r, V, 'call')
    put_premium = bs_price(S, K, T, r, V, 'put')
    long_call_payoff = np.maximum(stock_prices - K, 0) - call_premium
    short_put_payoff = -(np.maximum(K - stock_prices, 0) - put_premium)
    net_payoff_synthetic_long = long_call_payoff + short_put_payoff
    be = K + (call_premium - put_premium)
    fig = plot_with_breakeven(stock_prices, net_payoff_synthetic_long, be, 'Synthetic Long Stock', 'Synthetic Long Stock Payoff Diagram', strikes=[(K, 'Strike')], return_fig=True)

elif option_strategy == 'Synthetic Short':
    call_premium = bs_price(S, K, T, r, V, 'call')
    put_premium = bs_price(S, K, T, r, V, 'put')
    short_call_payoff = -(np.maximum(stock_prices - K, 0) - call_premium)
    long_put_payoff = np.maximum(K - stock_prices, 0) - put_premium
    net_payoff_synthetic_short = short_call_payoff + long_put_payoff
    be = K - (call_premium - put_premium)
    fig = plot_with_breakeven(stock_prices, net_payoff_synthetic_short, be, 'Synthetic Short Stock', 'Synthetic Short Stock Payoff Diagram', strikes=[(K, 'Strike')], return_fig=True)

elif option_strategy == 'Bull Call Spread':
    K1 = K * K1_spread
    K2 = K * K3_spread
    call_premium_K1 = bs_price(S, K1, T, r, V, 'call')
    call_premium_K2 = bs_price(S, K2, T, r, V, 'call')
    long_call_payoff = np.maximum(stock_prices - K1, 0) - call_premium_K1
    short_call_payoff = -(np.maximum(stock_prices - K2, 0) - call_premium_K2)
    bull_spread_payoff = long_call_payoff + short_call_payoff
    be = K1 + (call_premium_K1 - call_premium_K2)
    fig = plot_with_breakeven(stock_prices, bull_spread_payoff, be, 'Bull Call Spread', 'Bull Call Spread Payoff Diagram', strikes=[(K1, 'Long Call'), (K2, 'Short Call')], return_fig=True)

elif option_strategy == 'Bear Put Spread':
    K1 = K * K1_spread
    K2 = K * K3_spread
    put_premium_K1 = bs_price(S, K1, T, r, V, 'put')
    put_premium_K2 = bs_price(S, K2, T, r, V, 'put')
    long_put_payoff = np.maximum(K1 - stock_prices, 0) - put_premium_K1
    short_put_payoff = -(np.maximum(K2 - stock_prices, 0) - put_premium_K2)
    bear_spread_payoff = long_put_payoff + short_put_payoff
    be = K1 - (put_premium_K1 - put_premium_K2)
    fig = plot_with_breakeven(stock_prices, bear_spread_payoff, be, 'Bear Put Spread', 'Bear Put Spread Payoff Diagram', strikes=[(K1, 'Long Put'), (K2, 'Short Put')], return_fig=True)

elif option_strategy == 'Covered Call':
    call_premium = bs_price(S, K, T, r, V, 'call')
    long_stock_payoff = stock_prices - S
    short_call_payoff = -(np.maximum(stock_prices - K, 0) - call_premium)
    covered_call_payoff = long_stock_payoff + short_call_payoff
    be = S - call_premium
    fig = plot_with_breakeven(stock_prices, covered_call_payoff, be, 'Covered Call', 'Covered Call Payoff Diagram', strikes=[(K, 'Call Strike')], return_fig=True)

elif option_strategy == 'Protective Put':
    put_premium = bs_price(S, K, T, r, V, 'put')
    long_stock_payoff = stock_prices - S
    long_put_payoff = np.maximum(K - stock_prices, 0) - put_premium
    protective_put_payoff = long_stock_payoff + long_put_payoff
    be = S + put_premium
    fig = plot_with_breakeven(stock_prices, protective_put_payoff, be, 'Protective Put', 'Protective Put Payoff Diagram', strikes=[(K, 'Put Strike')], return_fig=True)

elif option_strategy == 'Collar':
    K1 = K * K1_spread
    K2 = K * K3_spread
    put_premium = bs_price(S, K1, T, r, V, 'put')
    call_premium = bs_price(S, K2, T, r, V, 'call')
    long_stock_payoff = stock_prices - S
    long_put_payoff = np.maximum(K1 - stock_prices, 0) - put_premium
    short_call_payoff = -(np.maximum(stock_prices - K2, 0) - call_premium)
    collar_payoff = long_stock_payoff + long_put_payoff + short_call_payoff
    be = S + put_premium - call_premium
    fig = plot_with_breakeven(stock_prices, collar_payoff, be, 'Collar', 'Collar Strategy Payoff Diagram', strikes=[(K1, 'Put Strike'), (K2, 'Call Strike')], return_fig=True)

elif option_strategy == 'Straddle':
    call_premium = bs_price(S, K, T, r, V, 'call')
    put_premium = bs_price(S, K, T, r, V, 'put')
    long_call_payoff = np.maximum(stock_prices - K, 0) - call_premium
    long_put_payoff = np.maximum(K - stock_prices, 0) - put_premium
    straddle_payoff = long_call_payoff + long_put_payoff
    be1 = K + call_premium + put_premium
    be2 = K - (call_premium + put_premium)
    fig = plot_with_breakeven(stock_prices, straddle_payoff, [be1, be2], 'Straddle', 'Straddle Payoff Diagram', strikes=[(K, 'Strike')], return_fig=True)

elif option_strategy == 'Butterfly Spread':
    K1 = K * K1_spread
    K2 = K * K2_spread
    K3 = K * K3_spread
    call_premium_K1 = bs_price(S, K1, T, r, V, 'call')
    call_premium_K2 = bs_price(S, K2, T, r, V, 'call')
    call_premium_K3 = bs_price(S, K3, T, r, V, 'call')
    long_call_K1 = np.maximum(stock_prices - K1, 0) - call_premium_K1
    short_2calls_K2 = -2 * (np.maximum(stock_prices - K2, 0) - call_premium_K2)
    long_call_K3 = np.maximum(stock_prices - K3, 0) - call_premium_K3
    butterfly_payoff = long_call_K1 + short_2calls_K2 + long_call_K3
    net_premium = call_premium_K1 - 2 * call_premium_K2 + call_premium_K3
    be1 = K1 + net_premium
    be2 = K3 - net_premium
    fig = plot_with_breakeven(stock_prices, butterfly_payoff, [be1, be2], 'Butterfly Spread', 'Butterfly Spread Payoff Diagram', strikes=[(K1, 'Buy Call'), (K2, 'Sell 2 Calls'), (K3, 'Buy Call')], return_fig=True)

elif option_strategy == 'Iron Condor':
    K0 = K * K1_spread
    K1_ = K * K2_spread
    K2_ = K * K3_spread
    K3_ = K * K4_spread
    put_premium_K0 = bs_price(S, K0, T, r, V, 'put')
    put_premium_K1 = bs_price(S, K1_, T, r, V, 'put')
    call_premium_K2 = bs_price(S, K2_, T, r, V, 'call')
    call_premium_K3 = bs_price(S, K3_, T, r, V, 'call')
    long_put_K0 = np.maximum(K0 - stock_prices, 0) - put_premium_K0
    short_put_K1 = -(np.maximum(K1_ - stock_prices, 0) - put_premium_K1)
    short_call_K2 = -(np.maximum(stock_prices - K2_, 0) - call_premium_K2)
    long_call_K3 = np.maximum(stock_prices - K3_, 0) - call_premium_K3
    iron_condor_payoff = long_put_K0 + short_put_K1 + short_call_K2 + long_call_K3
    net_credit = (put_premium_K1 - put_premium_K0) + (call_premium_K2 - call_premium_K3)
    be1 = K1_ - net_credit
    be2 = K2_ + net_credit
    fig = plot_with_breakeven(stock_prices, iron_condor_payoff, [be1, be2], 'Iron Condor', 'Iron Condor Payoff Diagram', strikes=[(K0, 'Buy Put'), (K1_, 'Sell Put'), (K2_, 'Sell Call'), (K3_, 'Buy Call')], return_fig=True)

elif option_strategy == 'Calendar Spread':
    K_cal = K
    T_short = T / 2
    T_long = T
    call_premium_short = bs_price(S, K_cal, T_short, r, V, 'call')
    call_premium_long = bs_price(S, K_cal, T_long, r, V, 'call')
    remaining_T = T_long - T_short
    long_call_value_at_Tshort = np.array([bs_price(SP, K_cal, remaining_T, r, V, 'call') for SP in stock_prices])
    short_call_payoff = -(np.maximum(stock_prices - K_cal, 0) - call_premium_short)
    calendar_spread_payoff = long_call_value_at_Tshort + short_call_payoff - call_premium_long
    fig = plot_with_breakeven(stock_prices, calendar_spread_payoff, K_cal, 'Calendar Spread', 'Calendar Spread Payoff Diagram', strikes=[(K_cal, 'Strike')], return_fig=True)

else:
    print("Invalid option strategy!")

# --- Export to Excel ---
if fig is not None:
    with tempfile.NamedTemporaryFile(suffix='.png', delete=False) as tmpfile:
        fig.savefig(tmpfile.name, format='png')
        tmpfile_path = tmpfile.name
    plt.close(fig)  # Close AFTER saving

    # Remove previous chart if it exists (robust fix, safe by name)
    for i in range(len(sht.pictures)):
        try:
            pic = sht.pictures[i]
            if hasattr(pic, "name") and pic.name == 'OptionChart':
                pic.delete()
        except Exception as e:
            continue

    # Add to Excel
    sht.pictures.add(tmpfile_path, name='OptionChart', left=sht.range('A21').left, top=sht.range('A21').top)

    # Clean up
    os.remove(tmpfile_path)

# --- Additional: Volatility Skew and Surface Plots (export to Excel) ---

# Get ticker symbol from Excel (assuming it's in cell 'B1')
ticker_symbol = sht.range('B1').value  # e.g., 'AAPL'
ticker = yf.Ticker(ticker_symbol)

expiration = sht.range('P20').value  # Get the actual value (string) from the cell
option_chain = ticker.option_chain(expiration)
calls = option_chain.calls

strikes = calls['strike'].reset_index(drop=True).values
implied_vols = calls['impliedVolatility'].reset_index(drop=True).values

# Implied Volatility Skew Plot
fig_skew, ax_skew = plt.subplots(figsize=(8, 5))
ax_skew.plot(strikes, implied_vols, marker='o')
ax_skew.set_title(f'Implied Volatility Skew for {ticker_symbol} {expiration}')
ax_skew.set_xlabel('Strike Price')
ax_skew.set_ylabel('Implied Volatility')
ax_skew.grid()

with tempfile.NamedTemporaryFile(suffix='.png', delete=False) as tmpfile:
    fig_skew.savefig(tmpfile.name, format='png')
    tmpfile_path = tmpfile.name
plt.close(fig_skew)

# Remove previous IVSkewChart if it exists (robust fix, safe by name)
for i in range(len(sht.pictures)):
    try:
        pic = sht.pictures[i]
        if hasattr(pic, "name") and pic.name == 'IVSkewChart':
            pic.delete()
    except Exception as e:
        continue

# Add to Excel
sht.pictures.add(tmpfile_path, name='IVSkewChart', left=sht.range('L21').left, top=sht.range('L21').top)
os.remove(tmpfile_path)

# Implied Volatility Surface Plot
strikes_surface = np.arange(80, 121, 5)  # Strike prices
maturities = np.array([10, 30, 60, 90, 180, 360])  # Days to expiration
X, Y = np.meshgrid(strikes_surface, maturities)
base_vol = 0.20
vol_surface = (base_vol +
               0.002 * (100 - X) +
               0.0005 * (Y - 30) +
               0.01 * np.sin((X-100)/10)*np.cos((Y-30)/50))

fig_surface = plt.figure(figsize=(10, 6))
ax_surface = fig_surface.add_subplot(111, projection='3d')
ax_surface.plot_surface(X, Y, vol_surface, cmap='viridis', edgecolor='none')
ax_surface.set_title('Implied Volatility Surface')
ax_surface.set_xlabel('Strike Price')
ax_surface.set_ylabel('Days to Expiry')
ax_surface.set_zlabel('Implied Volatility')

with tempfile.NamedTemporaryFile(suffix='.png', delete=False) as tmpfile:
    fig_surface.savefig(tmpfile.name, format='png')
    tmpfile_path = tmpfile.name
plt.close(fig_surface)

# Remove previous IVSurfaceChart if it exists (robust fix, safe by name)
for i in range(len(sht.pictures)):
    try:
        pic = sht.pictures[i]
        if hasattr(pic, "name") and pic.name == 'IVSurfaceChart':
            pic.delete()
    except Exception as e:
        continue

# Add to Excel
sht.pictures.add(tmpfile_path, name='IVSurfaceChart', left=sht.range('U21').left, top=sht.range('U21').top)
os.remove(tmpfile_path)